In [ ]:
# get live chat

import pytchat
# test lofi girl live
# https://www.youtube.com/watch?v=jfKfPfyJRdk

# test moonrabbit: https://www.youtube.com/watch?v=op_CtC7mJms

chat = pytchat.create(video_id="jfKfPfyJRdk")
while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")

In [6]:
import glob

# testing playing video with choice - choices are numbers 1-5

def make_videos_dict():
    files = glob.glob('/Volumes/My Passport/UAL_Thesis/raw_videos/*')
    files = [f.split('/')[-1] for f in files]
    files_dict = {str(i+1): value for i, value in enumerate(files)}

    return files_dict

def test_choose_videos(i , videos_dict):
    return videos_dict[i]

import cv2
import time

# play video according to live chat command using cv2 
raw_videos_path = '/Volumes/My Passport/UAL_Thesis/raw_videos'
videos_dict = make_videos_dict()

def play_video(video_path, play_time=5):
    cap = cv2.VideoCapture(video_path)

    start_time = time.time()
    while(cap.isOpened() and time.time() - start_time < play_time):
        ret, frame = cap.read()
        if ret == True:
            cv2.imshow('Frame', frame)
            # Press Q on keyboard to exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    cap.release()
    cv2.destroyAllWindows()

# while True:
#     video_file_input = input("Enter the video file path choice to play or 'q' to quit: ")
#     if video_file_input.lower() == 'q':
#         break
#     else:
#         video_file = raw_videos_path + '/'+test_choose_videos(video_file_input, videos_dict)
#         print(video_file)
#         play_video(video_file)


In [7]:
# choose videos using live chat

# video_id = 
import pytchat

chat = pytchat.create(video_id="CUscoOh1zcI")
while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
        video_file = raw_videos_path + '/'+test_choose_videos(c.message, videos_dict)
        print(video_file)
        play_video(video_file)

2023-08-07 23:47:22 [Moon Rabbit]- 1
/Volumes/My Passport/UAL_Thesis/raw_videos/idle_bob_1.MOV
2023-08-07 23:47:38 [Moon Rabbit]- 2
/Volumes/My Passport/UAL_Thesis/raw_videos/idle_bob_2.MOV
2023-08-07 23:47:50 [Moon Rabbit]- 3
/Volumes/My Passport/UAL_Thesis/raw_videos/jump_out_in.MOV
2023-08-07 23:48:00 [Moon Rabbit]- 4
/Volumes/My Passport/UAL_Thesis/raw_videos/step_f_b.MOV


In [ ]:
# test obs websocket

import cv2
import pytchat
from obswebsocket import obsws, requests
import pygetwindow as gw
import pyautogui
import time

# Connect to OBS
# host = 
# port = 4444  # Default port for obs-websocket
# pwd = 
ws = obsws(host, port,pwd)
ws.connect()



# don't forget to manually select the live stream in OBS first before starting to stream
# also stream will not be present if previous livestream edit window is not closed on youtube studios

    # Start streaming
ws.call(requests.StartStreaming())





In [ ]:
# test obs websocket
# live stream url: https://youtu.be/nPdEhg5zNMU

import cv2
import pytchat
from obswebsocket import obsws, requests
import pygetwindow as gw
import pyautogui
import time

# Connect to OBS
# host = 
# port = 4444  # Default port for obs-websocket
# pwd = 
ws = obsws(host, port,pwd)
ws.connect()

# List all profiles (broadcasts)
# current_settings = ws.call(requests.GetStreamSettings())
# print(current_settings)

# create scene
scene_request = requests.CreateScene()
scene_request.scene_name = "OpenCV_Scene"
scene_name =  "OpenCV_Scene"
ws.call(scene_request)


# Add a "Window Capture" source to the scene
source_name = "OpenCV_Window_Capture"
window_name = "Frame"  # Name of the OpenCV window (as per cv2.imshow('Frame', frame))
source_request = requests.CreateSource()
source_request.sourceName = source_name
source_request.sourceType = "window_capture"
source_request.sourceSettings = {
    "window": window_name + ":.*",  # This regex ensures it matches the window title even if there are additional characters
    "window_match_priority": 1  # Match by window title
}
ws.call(source_request)

# Add the created source to the scene
ws.call(requests.AddSceneItem(scene_name, source_name, -1))




# don't forget to manually select the live stream in OBS first before starting to stream
# also stream will not be present if previous livestream edit window is not closed on youtube studios

    # Start streaming
ws.call(requests.StartStreaming())



chat = pytchat.create(video_id="nPdEhg5zNMU")
while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
        video_file = raw_videos_path + '/'+test_choose_videos(c.message, videos_dict)
        print(video_file)
        play_video(video_file)

ws.disconnect()

In [4]:
import cv2
import pytchat
from obswebsocket import obsws, requests
import pygetwindow as gw
import pyautogui
import time

# Connect to OBS
host = "192.168.68.125"
port = 4444  # Default port for obs-websocket
pwd = 'uqbZrtM7tZBjjBh1'
ws = obsws(host, port, pwd)
ws.connect()

# Create an empty window named 'Frame'to add to OBS scene
cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)

# Create a new scene
scene_request = requests.CreateScene()
scene_request.scene_name = "Scene"
scene_name = "Scene"
ws.call(scene_request)

# Add a "Window Capture" source to the scene
source_name = "OpenCV_Window_Capture"
window_name = "Frame"  # Name of the OpenCV window
source_request = requests.CreateSource()
source_request.sourceName = source_name
source_request.sourceType = "window_capture"
source_request.sourceSettings = {
    "window": window_name ,  # Match the window title with regex
    "window_match_priority": 1  # Match by window title
}
ws.call(source_request)

# Add the created source to the scene
add_item_request = requests.AddSceneItem()
add_item_request.scene_name = scene_name
add_item_request.source_name = source_name
add_item_request.set_order = -1
ws.call(add_item_request)

# Start streaming
ws.call(requests.StartStreaming())

# somehow get the stream share link from obs

chat = pytchat.create(video_id="geXBNzhtCsM")
while chat.is_alive():
    for c in chat.get().sync_items():
        print(f"{c.datetime} [{c.author.name}]- {c.message}")
        if c.message.lower() == 'q':
            break
        elif c.message.isdigit() and c.message in videos_dict:
            video_file = raw_videos_path + '/' + test_choose_videos(c.message, videos_dict)
            print(video_file)
            play_video(video_file)

ws.disconnect()


In [2]:
import cv2
cv2.namedWindow('Frame', cv2.WINDOW_NORMAL)

In [ ]:
# test obs websocket

import cv2
import pytchat
from obswebsocket import obsws, requests
import pygetwindow as gw
import pyautogui
import time

# Connect to OBS
host = "192.168.68.125"
port = 4455  # Default port for obs-websocket
pwd = 'uqbZrtM7tZBjjBh1'
ws = obsws(host, port,pwd)
ws.connect()

# Function to set up OBS
def setup_obs():
    # Create a new Scene 'OpenCV_Scene'
    ws.call(requests.CreateScene("OpenCV_Scene"))

    # Set the new scene as the current scene
    ws.call(requests.SetCurrentScene("OpenCV_Scene"))
    
    # Create a new 'Window Capture' source and add it to 'OpenCV_Scene'
    ws.call(requests.CreateSource("Window Capture", "OpenCV_Capture", {}))
    ws.call(requests.AddSceneItem("OpenCV_Scene", "OpenCV_Capture", "OpenCV Window"))

# Start streaming function
def start_streaming():
    ws.call(requests.StartStreaming())

# OpenCV video feed
def opencv_feed():
    cap = cv2.VideoCapture(0)
    cv2.namedWindow('OpenCV Window', cv2.WINDOW_NORMAL)
    
    while True:
        ret, frame = cap.read()
        cv2.imshow('OpenCV Window', frame)  # 'OpenCV Window' is the name of the window

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Position the OpenCV window for consistent capture
def position_opencv_window():
    window = gw.getWindowsWithTitle('OpenCV Window')[0]
    window.moveTo(100, 100)
    window.resize(640, 480)  # This is just a standard resolution; adjust as needed

def set_active_scene(ws, scene_name):
    ws.call(requests.SetCurrentScene(scene_name))

# Set everything up and start streaming
setup_obs()
opencv_feed()
time.sleep(2)  # A small delay to ensure the OpenCV window is ready
position_opencv_window()
set_active_scene(ws, "Moon Rabbit Live Stream")
start_streaming()

# Disconnect from OBS when done
ws.disconnect()


In [ ]:
# import cv2
# # use this to test cv2 play video
# # for macos use python 3.10 because otherwise gstreamer pipeline doesn't work with opencv, don't know why

# # Specify the path to your .MOV file
# # video_path = '/Volumes/My Passport/UAL_Thesis/raw_videos/idle_bob_1.MOV'
# video_path = 'idle_bob_2.MOV'
# # video_path = 'output.MOV'

# # Create a VideoCapture object
# cap = cv2.VideoCapture(video_path)

# # Check if video opened successfully
# if (cap.isOpened() == False): 
#   print("Error opening video file")

# # Read and display video frames until video is completed or user quits by pressing Q
# while(cap.isOpened()):
  
#   # Capture frame-by-frame
#   ret, frame = cap.read()

#   if ret == True:
#     # Display the resulting frame
#     cv2.imshow('Frame', frame)

#     # Press Q on keyboard to exit
#     if cv2.waitKey(25) & 0xFF == ord('q'):
#       break

#   # Break the loop if the video is finished
#   else: 
#     break

# # When everything is done, release the video capture and video write objects
# cap.release()

# # Close all the frames
# cv2.destroyAllWindows()
